In [ ]:
'''
Programme to process data for the ASHRAE Great Energy Predictor 3
Author: @SpencerAndTheMatt


'''

In [1]:
# Define imports
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
'''
README
If your files are saved elsewhere (i.e not google drive) then you will
need to update this section. Keep training data as train_data, weather
train data as weather_train_data, etc.

Do not change the variable names, the programme won't like it
'''
# Acquire data from google drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

# Training data
train_data = pd.read_csv('/content/drive/My Drive/train.csv') # Change for training data
weather_train_data = pd.read_csv('/content/drive/My Drive/weather_train.csv') # Change for weather train data

# Building data
building_data = pd.read_csv('/content/drive/My Drive/building_metadata.csv') # Change for building metadata

# Testing data
test_data = pd.read_csv('/content/drive/My Drive/test.csv') # Change for testing data
weather_test_data = pd.read_csv('/content/drive/My Drive/weather_test.csv') # Change for weather test data

Mounted at /content/drive


In [3]:
# Convert data to feather to reduce memory
# Train data first
train_data.to_feather('train_data.feather')
weather_train_data.to_feather('weather_train_data.feather')

# Building data
building_data.to_feather('building_data.feather')

# Test data
test_data.to_feather('test_data.feather')
weather_test_data.to_feather('weather_test_data.feather')

In [4]:
# Return data to pandas DataFrame
# Train data first
train_data = pd.read_feather('train_data.feather')
weather_train_data = pd.read_feather('weather_train_data.feather')

# Building data
building_data = pd.read_feather('building_data.feather')

# Test data
test_data = pd.read_feather('test_data.feather')
weather_test_data = pd.read_feather('weather_test_data.feather')

In [5]:
# This function helps in optimizing the memory used by the dataframes by by modifying/altering thedatatype of each column.
# This method is inspired from the kaggle notebook titled: load data (reduce memory usage)
# Which is found at: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df, df_name):

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:

        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2

    print('Memory usage of {} is reduced by {:.2f} %. Usage dropped from {:.2f} MB to {:.2f} MB.'.format(df_name, (100 * (start_mem - end_mem) / start_mem), start_mem, end_mem))
    
    return df

In [6]:
# Reduce memory usage of data
# Training data first
train_data = reduce_mem_usage(train_data, 'Train Data')
weather_train_data = reduce_mem_usage(weather_train_data, 'Weather Train Data')

# Building data
building_data = reduce_mem_usage(building_data, 'Building Data')

# Test data
test_data = reduce_mem_usage(test_data, 'Test Data')
weather_test_data = reduce_mem_usage(weather_test_data, 'Weather Test Data')

Memory usage of Train Data is reduced by 71.82 %. Usage dropped from 616.95 MB to 173.84 MB.
Memory usage of Weather Train Data is reduced by 73.06 %. Usage dropped from 9.60 MB to 2.59 MB.
Memory usage of Building Data is reduced by 73.88 %. Usage dropped from 0.07 MB to 0.02 MB.
Memory usage of Test Data is reduced by 71.82 %. Usage dropped from 1272.51 MB to 358.53 MB.
Memory usage of Weather Test Data is reduced by 73.04 %. Usage dropped from 19.04 MB to 5.13 MB.


In [7]:
# Merge data together
# Merge training data
train = train_data.merge(building_data, on = 'building_id', how = 'left')
train = train.merge(weather_train_data, on = ['site_id', 'timestamp'], how = 'left')

# Merge testing data
test = test_data.merge(building_data, on = 'building_id', how = 'left')
test = test.merge(weather_test_data, on = ['site_id', 'timestamp'], how = 'left')

In [8]:
# Delete old variables - weather_test_data, weather_train_data, building_data
del weather_test_data, weather_train_data, building_data

In [9]:
# Put timestamp into form Hour, Day, Month, Year
def breakdown_timestamp(dataframe):
    
    dataframe['timestamp']= pd.to_datetime(dataframe['timestamp'])

    dataframe['hour']= np.uint8(dataframe['timestamp'].dt.hour)
    
    dataframe['day']= np.uint16(dataframe['timestamp'].dt.day)
    dataframe['day_of_week']= np.uint8(dataframe['timestamp'].dt.dayofweek)
    dataframe['day_of_year']= np.uint16(dataframe['timestamp'].dt.dayofyear)

    dataframe['month']= np.uint8(dataframe['timestamp'].dt.month)

    dataframe['year']= np.uint16(dataframe['timestamp'].dt.year)
    
    return dataframe

In [10]:
# Call breakdown timestamp on both train and test datasets
# Train first (If you run both in the same cell colab will crash)
train = breakdown_timestamp(train)

In [11]:
# Now test
test = breakdown_timestamp(test)

In [12]:
# Quick inspection of data
#train.head()
test.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,day_of_week,day_of_year,month,year
0,0,0,0,2017-01-01,0,Education,7432,2008.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
1,1,1,0,2017-01-01,0,Education,2720,2004.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
2,2,2,0,2017-01-01,0,Education,5376,1991.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
3,3,3,0,2017-01-01,0,Education,23685,2002.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
4,4,4,0,2017-01-01,0,Education,116607,1975.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017


In [13]:
# Apply log1p trainsformation to Meter Reading and Square Feet 
# Apply to train
train['meter_reading'] = np.log1p(train['meter_reading'])
train['square_feet'] = np.log1p(train['square_feet'])

# Apply to test (But only to square feet as no meter reading in test)
test['square_feet'] = np.log1p(test['square_feet'])

In [14]:
test.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,day_of_week,day_of_year,month,year
0,0,0,0,2017-01-01,0,Education,8.913685,2008.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
1,1,1,0,2017-01-01,0,Education,7.908755,2004.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
2,2,2,0,2017-01-01,0,Education,8.589886,1991.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
3,3,3,0,2017-01-01,0,Education,10.072639,2002.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017
4,4,4,0,2017-01-01,0,Education,11.666573,1975.0,NaN,17.796875,...,NaN,1021.5,100.0,3.599609,0,1,6,1,1,2017


In [15]:
# Data prep + feature engineering
# Remove zero values
zero_meter = list(train[train['meter_reading'] == 0].index)
train.drop(zero_meter, axis = 0, inplace = True)

# No meter reading in test so can ignore

In [16]:
# Drop columns with >50% missing values
train.dropna(axis = 1, thresh = len(train) * 0.5, inplace = True)
test.dropna(axis = 1, thresh = len(test) * 0.5, inplace = True)

In [17]:
# Fill missing values with median
for i in train.columns:
  try:
    train[i].fillna(train[i].median(), inplace = True)
  except:
    pass

for i in test.columns:
  try:
    test[i].fillna(test[i].median(), inplace = True)
  except:
    pass

In [18]:
# Add new features in the possibly vain hope of better results
train['season'] = train['timestamp'].apply(lambda x: 'Spring' if x.month==3 or x.month==4 or x.month==5 else 
                                                  'Summer' if x.month==6 or x.month==7 or x.month==8 else 
                                                  'Autumn' if x.month==9 or x.month==10 or x.month==11 else 
                                                  'Winter')

test['season'] = test['timestamp'].apply(lambda x: 'Spring' if x.month==3 or x.month==4 or x.month==5 else 
                                                  'Summer' if x.month==6 or x.month==7 or x.month==8 else 
                                                  'Autumn' if x.month==9 or x.month==10 or x.month==11 else 
                                                  'Winter')


train['isDayTime'] = train['timestamp'].apply(lambda x: 1 if x.hour >=6 and x.hour <=18 else 0)
test['isDayTime'] = test['timestamp'].apply(lambda x: 1 if x.hour >=6 and x.hour <=18 else 0)

In [19]:
# Encode categorical data
categorical_features = ['primary_use', 'season']
encoder = preprocessing.LabelEncoder()

for i in categorical_features:
  train[i] = encoder.fit_transform(train[i])
  test[i] = encoder.fit_transform(test[i])

In [20]:
# Remove timestamp
train, test = train.drop(['timestamp'], axis = 1), test.drop(['timestamp'], axis = 1)

In [21]:
# Reduce memory usage (again)
reduced_train_data = reduce_mem_usage(train, 'Train Data')
reduced_test_data = reduce_mem_usage(test, 'Test Data')

Memory usage of Train Data is reduced by 36.62 %. Usage dropped from 1241.96 MB to 787.16 MB.
Memory usage of Test Data is reduced by 33.80 %. Usage dropped from 2823.38 MB to 1869.00 MB.


In [22]:
# Delete old variables
del train, test, encoder, categorical_features, zero_meter

In [23]:
# Remove least important features generated by feature selection
training_processed = reduced_train_data[['building_id','square_feet','primary_use','meter','air_temperature','day_of_year','hour','isDayTime','day_of_week', 'meter_reading']]
testing_processed = reduced_test_data[['building_id','square_feet','primary_use','meter','air_temperature','day_of_year','hour','isDayTime','day_of_week']]

In [25]:
# Export both training_processed and testing_processed to csv files (To drive)
training_processed.to_csv('/content/drive/My Drive/training_data_processed.csv')
testing_processed.to_csv('/content/drive/My Drive/testing_data_processed.csv')

In [27]:
# Export both training_processed and testing_processed to feather files (To drive)
training_processed.reset_index().to_feather('/content/drive/My Drive/training_data_processed_feather.feather')
testing_processed.reset_index().to_feather('/content/drive/My Drive/testing_data_processed_feather.feather')